In [ ]:
import numpy as np
import tensorflow as tf
import cv2

class_names = ['airport_image_logmel', 'bus_image_logmel', 'metro_image_logmel', 'park_image_logmel', 'shopping_mall_image_logmel']  # your class list

interpreter = tf.lite.Interpreter(model_path="bc_resnet_40_int8.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input image (uint8 expected for int8 model)
img = cv2.imread(" ")  # Your file
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224, 224))

# Quantize input image using input scale and zero point
input_scale, input_zero_point = input_details[0]['quantization']
input_data = img.astype(np.float32) / 255.0  # normalize first if needed
input_data = input_data / input_scale + input_zero_point
input_data = np.clip(input_data, 0, 255).astype(np.uint8)
input_data = np.expand_dims(input_data, axis=0)

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Dequantize output
output_data = interpreter.get_tensor(output_details[0]['index'])
output_scale, output_zero_point = output_details[0]['quantization']
output_float = output_scale * (output_data.astype(np.float32) - output_zero_point)

predicted_index = np.argmax(output_float[0])
predicted_label = class_names[predicted_index]

print(f"Predicted class: {predicted_label}")
